In [1]:
import numpy as np
from matplotlib import pyplot as plt 
%matplotlib inline

In [2]:
import astropy
from astropy.io import fits

from astropy.coordinates import SkyCoord

In [3]:
hdu_z = fits.open('./data/decaps_l0_b0_hdu08_z')

In [4]:
hdu_i = fits.open('./data/decaps_l0_b0_hdu08_i')

In [5]:
hdu_z[1].header

XTENSION= 'IMAGE   '           / Image extension                                
BITPIX  =                  -32 / Bits per pixel                                 
NAXIS   =                    2 / Number of axes                                 
NAXIS1  =                 2046 / Axis length                                    
NAXIS2  =                 4094 / Axis length                                    
PCOUNT  =                    0 / No 'random' parameters                         
GCOUNT  =                    1 / Only one group                                 
ORIGIN  = 'NOAO-IRAF FITS Image Kernel July 2003' / FITS file originator        
EXTNAME = 'S21     '           / Extension name                                 
IRAF-TLM= '2016-08-20T06:14:52' / Time of last modification                     
INHERIT =                    T / Inherits global header                         
DATE    = '2016-08-20T06:14:04' / Date FITS file was generated                  
WCSAXES =                   

### sky backgrounds

In [6]:
hdu_z[1].header['SKYBRITE']

680.62

In [7]:
hdu_i[1].header['SKYBRITE']

476.77295

### PSF

In [8]:
hdu_z[1].header['FWHM']

4.207338

In [9]:
hdu_i[1].header['FWHM']

4.546045

In [10]:
hdu_z

[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7faa06a83e50>, <astropy.io.fits.hdu.compressed.CompImageHDU object at 0x7faa069dbcd0>]

In [11]:
def get_gauss_psf_params_from_fwhm(fwhm):
    # half light radius
    r = fwhm / 2
    
    psf_params_array = np.zeros((1, 6)) 
    sigma2 = r**2 / (2 * np.log(2))
    b = 1e-16
    p0 = 1e-16

    psf_params_array[0, 0] = np.log(sigma2)
    psf_params_array[0, 4] = np.log(b)
    psf_params_array[0, 5] = np.log(p0)
    
    return(psf_params_array)

In [12]:
psf_params_z = get_gauss_psf_params_from_fwhm(hdu_z[1].header['FWHM'])
print(psf_params_z)

[[  1.16073167   0.           0.           0.         -36.84136149
  -36.84136149]]


In [13]:
np.save('./psf/zband_psf_gauss.npy', psf_params_z)

In [14]:
psf_params_i = get_gauss_psf_params_from_fwhm(hdu_i[1].header['FWHM'])
print(psf_params_i)

[[  1.31558663   0.           0.           0.         -36.84136149
  -36.84136149]]


In [15]:
np.save('./psf/iband_psf_gauss.npy', psf_params_i)

# reasonable init for psf optimization

In [16]:
def get_init_psf_params_from_fwhm(fwhm):
    # half light radius
    r = fwhm / 2
    
    psf_params_array = np.zeros((1, 6)) 
    sigma2 = r**2 / (2 * np.log(2))
    b = 0.5
    p0 = 0.5
    
    psf_params_array[0, 0] = np.log(sigma2)
    psf_params_array[0, 1] = np.log(sigma2 * 2)
    
    psf_params_array[0, 2] = np.log(1)
    psf_params_array[0, 3] = np.log(1)
    
    psf_params_array[0, 4] = np.log(b)
    psf_params_array[0, 5] = np.log(p0)
    
    return(psf_params_array)

In [17]:
psf_params_z = get_init_psf_params_from_fwhm(hdu_z[1].header['FWHM'])
print(psf_params_z)

[[ 1.16073167  1.85387885  0.          0.         -0.69314718 -0.69314718]]


In [18]:
np.save('./psf/zband_psf_init.npy', psf_params_z)

In [19]:
psf_params_i = get_init_psf_params_from_fwhm(hdu_i[1].header['FWHM'])
print(psf_params_i)

[[ 1.31558663  2.00873381  0.          0.         -0.69314718 -0.69314718]]


In [20]:
np.save('./psf/iband_psf_init.npy', psf_params_i)